# Imports

In [1]:
# Standard library imports
import os
from datetime import datetime

# Third-party library imports
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from skimage.metrics import structural_similarity as ssim
from torch.distributions import MultivariateNormal
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image
from tqdm import tqdm

# Local application/library specific imports

from objectives import compute_elbo_dist as compute_elbo#, compute_elbo_mmd,compute_elbo_dreg
# from test_functions import calculate_fid_routine
from utils import Constants, initialize_weights, unpack_data_biked
from test_functions_torch import calculate_fid_routine

True


In [2]:
resolution = 64
n_channels = 3

In [3]:
use_resnet = True
if use_resnet:
    from models.gmrf_mvae_biked import Biked_GMMVAE
else:
    from models.gmrf_mvae_biked_simpler import Biked_GMMVAE

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [5]:
torch.manual_seed(0)

In [6]:
torch.cuda.manual_seed_all(0)

In [7]:
project_path = '../'

# Utils

In [8]:
import matplotlib.pyplot as plt
import torch
from torchvision.utils import make_grid

def save_unconditional_generations(unconditional_generations, save_path):
    """
    Create and save a grid for unconditional generations.
    
    Args:
        unconditional_generations (list of torch.Tensor): A list of 6 tensors, 
            each of shape [N, 1, resolution, resolution].
        save_path (str): The file path to save the resulting image.
    """
    # Ensure all tensors are on CPU
    # generations = [gen.cpu() for gen in unconditional_generations]
    generations = [torch.mean(gen.cpu(), dim=1, keepdim=True) for gen in unconditional_generations]  # Convert [k,3,64,64] → [k,1,64,64]
    superposition = torch.stack(generations, dim=0).sum(dim=0)  # Summing grayscale images


    # 1) Create the superposition by summing across the 6 modalities
    #    (shape: [N, 1, resolution, resolution])
    # superposition = torch.stack(generations, dim=0).sum(dim=0)
    
    # 2) Append this superposition as the 7th "modality"
    all_generations = generations + [superposition]

    # 3) Prepare a grid for each modality / system component
    grids = []
    for tensor in all_generations:
        # Flatten the batch for a single column
        tensor = tensor.view(-1, 1, tensor.shape[-2], tensor.shape[-1]) 
        grid = make_grid(tensor, nrow=1, padding=2, normalize=False)
        grids.append(grid)
    
    # 4) Combine all grids into a single image (7 columns now)
    full_grid = torch.cat(grids, dim=2)  # Concatenate along width

    # 5) Convert to numpy for plotting
    full_grid_image = full_grid.permute(1, 2, 0).numpy()

    # 6) Plot and save
    #    Figure size might need adjusting depending on your batch size
    plt.figure(figsize=(7, len(unconditional_generations[0]) * 2))
    plt.imshow(full_grid_image, cmap='gray', vmin=0, vmax=full_grid_image.max())
    plt.axis("off")
    plt.title("Unconditional Generations (6 components + superposition)")
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()

    print(f"Unconditional generation grid (with superposition) saved to {save_path}")


In [9]:
import matplotlib.pyplot as plt
import torch
from torchvision.utils import make_grid

def save_cross_generations(cross_generations, save_path):
    """
    Plot and save a 5x5 grid of cross-generated images.

    Args:
        cross_generations (list of lists): A 5x5 nested list where each element is a torch.Tensor 
                                           of shape [1, 3, 28, 28], located on GPU.
        save_path (str): The file path where the grid image will be saved.
    """
    # Initialize a 5x5 grid to store the images
    grid = []
    for row in cross_generations:
        row_images = torch.cat([img.squeeze(0).cpu() for img in row], dim=2)  # Concatenate images in the row
        grid.append(row_images)

    # Combine rows into one big grid
    full_grid = torch.cat(grid, dim=1)

    # Use make_grid to arrange the grid
    grid_image = make_grid(full_grid.unsqueeze(0), nrow=1, padding=2, normalize=False).permute(1, 2, 0)

    # Plot the grid using matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(grid_image)
    plt.axis("off")
    plt.title("Cross-Generation Results")

    # Save the grid image
    plt.savefig(save_path, bbox_inches="tight")
    # plt.show()
    plt.close()

    print(f"Cross-generation grid saved to {save_path}")


In [10]:
def plot_modalities(modalities, n_images):
    """
    Plots a grid of n_images x 5 to compare modalities of the same object.
    
    :param modalities: A list of 5 batches with shape [Batch_size, 3, 28, 28].
    :param n_images: Number of images to display from each modality.
    """
    # Set up the subplot grid
    fig, axes = plt.subplots(n_images, 5, figsize=(15, 3 * n_images))
    
    for i in range(n_images):
        for j in range(5):
            ax = axes[i][j] if n_images > 1 else axes[j]
            # Extract the i-th image from the j-th modality
            img = modalities[j][i].detach().cpu().numpy().T
            ax.imshow(img)
            ax.axis('off')  # Turn off axis
            if i == 0:
                # Label the top of each column with the modality number
                ax.set_title(f'Modality {j+1}')
    
    plt.tight_layout()
    plt.show()

In [11]:
def plot_cond_modalities(cond, modalities, n_images):
    """
    Plots a grid of n_images x 5 to compare modalities of the same object.
    
    :param modalities: A list of 5 batches with shape [Batch_size, 3, 28, 28].
    :param n_images: Number of images to display from each modality.
    """
    # Set up the subplot grid
    fig, axes = plt.subplots(n_images, 6, figsize=(15, 3 * n_images))
    
    for i in range(n_images):
        for j in range(6):
            ax = axes[i][j] if n_images > 1 else axes[j]
            if j==0:
                img = cond[i].detach().cpu().numpy().T
            # Extract the i-th image from the j-th modality
            else:
                img = modalities[j-1][i].detach().cpu().numpy().T
            ax.imshow(img)
            ax.axis('off')  # Turn off axis
            if i == 0:
                if j==0:
                    # Label the top of each column with the modality number
                    ax.set_title('Condition')
                else:
                # Label the top of each column with the modality number
                    ax.set_title(f'Modality {j+1}')
    
    plt.tight_layout()
    
    plt.show()

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [13]:
def plot_training_curves(mean_losses, mean_log_px_zs, mean_kls, save_path=None):
    epochs = len(mean_losses)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 3, 1)
    plt.plot(range(epochs), mean_losses, label='Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss')

    plt.subplot(1, 3, 2)
    plt.plot(range(epochs), mean_log_px_zs, label='log p(x|z)')
    plt.xlabel('Epochs')
    plt.ylabel('log p(x|z)')
    plt.title('log p(x|z)')

    plt.subplot(1, 3, 3)
    plt.plot(range(epochs), mean_kls, label='KL Divergence')
    plt.xlabel('Epochs')
    plt.ylabel('KL Divergence')
    plt.title('KL Divergence')

    plt.tight_layout()
    if save_path!=None:
        # Save the grid image
        plt.savefig(save_path, bbox_inches="tight")
        # plt.show()
        plt.close()

    plt.show()

In [14]:
def plot_covariance_matrix(cov_matrix, block_sizes, off_diag=False, cmap='hot', grid_color='white', grid_linewidth=2, title='Absolute Values of Covariance Matrix with Blocks'):
    """
    Plots a heatmap of a covariance matrix with gridlines delineating the blocks.
    If off_diag is True, the diagonal blocks are covered with white squares.

    Parameters:
    - cov_matrix: 2D numpy array representing the covariance matrix.
    - block_sizes: List of integers representing the size of each square block.
    - off_diag: Boolean flag to cover diagonal blocks with white squares.
    - cmap: Colormap for the heatmap.
    - grid_color: Color of the gridlines.
    - grid_linewidth: Width of the gridlines.
    """
    plt.imshow(abs(cov_matrix), cmap=cmap, interpolation='nearest')
    plt.colorbar()

    # Determine the boundaries of the blocks
    block_boundaries = np.cumsum([0] + block_sizes)

    # Plot white squares on diagonal blocks if off_diag is True
    if off_diag:
        ax = plt.gca()
        start = 0
        for size in block_sizes:
            # Create a white rectangle
            rect = patches.Rectangle((start, start), size, size, linewidth=0, edgecolor=None, facecolor='white')
            ax.add_patch(rect)
            start += size

    # Draw gridlines based on block boundaries
    for boundary in block_boundaries:
        plt.axhline(boundary - 0.5, color=grid_color, linestyle='-', linewidth=grid_linewidth)
        plt.axvline(boundary - 0.5, color=grid_color, linestyle='-', linewidth=grid_linewidth)

    # Hide the axis
    plt.axis('off')
    plt.title(title)

    # Show the plot with gridlines
    plt.show()


In [15]:
def plot(modalities, n_images):
    """
    Plots a grid of n_images x 5 to compare modalities of the same object.
    
    :param modalities: A list of 5 batches with shape [Batch_size, 3, 28, 28].
    :param n_images: Number of images to display from each modality.
    """
    # Set up the subplot grid
    fig, axes = plt.subplots(n_images, 5, figsize=(15, 3 * n_images))
    
    for i in range(n_images):
        for j in range(5):
            ax = axes[i][j] if n_images > 1 else axes[j]
            # Extract the i-th image from the j-th modality
            img = modalities[j][i].T
            ax.imshow(img)
            ax.axis('off')  # Turn off axis
            if i == 0:
                # Label the top of each column with the modality number
                ax.set_title(f'Modality {j+1}')
    
    plt.tight_layout()
    plt.show()

In [16]:
def remove_diagonal_blocks(cov_matrix, block_sizes):
    """
    Returns a new matrix with the diagonal blocks set to zero.

    Parameters:
    - cov_matrix: 2D numpy array representing the covariance matrix.
    - block_sizes: List of integers representing the size of each square block.

    Returns:
    - A new covariance matrix with diagonal blocks set to zero.
    """
    new_cov_matrix = cov_matrix.copy()
    start = 0
    for block_size in block_sizes:
        end = start + block_size
        new_cov_matrix[start:end, start:end] = np.zeros((block_size, block_size))
        start = end
    return new_cov_matrix

In [17]:
def update_learning_rate(epoch, optimizer, base_lr, max_lr, warmup_epochs):
    """Updates the learning rate over a span of epochs.
    
    Args:
        epoch: Current epoch.
        base_lr: Initial learning rate at the start of warmup.
        max_lr: Target learning rate after warmup.
        warmup_epochs: Number of epochs over which warmup occurs.
    """
    if epoch < warmup_epochs:
        lr = base_lr + (max_lr - base_lr) * (epoch / warmup_epochs)
    else:
        lr = max_lr

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train

## Train function

In [18]:
def create_log_dir(epochs, beta, latent_dim, n_layers, hidden_dim, 
                   diagonal_transf, l1=False, is_mmd = False, custom=False):
    # Generate a unique directory name
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    with_resnet = 'resnet' if use_resnet else 'simpler_resnet'
    if is_mmd:
        log_and_model_dir = f'{project_path}/runs/gmrf/{with_resnet}/ep_{epochs}_beta_{beta}_lt{latent_dim}_n{n_layers}_hd{hidden_dim}_tf_{diagonal_transf}_mmd_custom_{custom}_{timestamp}'
    else:
        log_and_model_dir = f'{project_path}/runs/gmrf/{with_resnet}/ep_{epochs}_beta_{beta}_lt_{latent_dim}_n_{n_layers}_hd_{hidden_dim}_tf_{diagonal_transf}_{timestamp}'
    # Create the directory
    if l1:
        log_and_model_dir+='_l1'
    os.makedirs(log_and_model_dir, exist_ok=True)
    # Use this directory for both TensorBoard logs and saving models
    return log_and_model_dir

In [19]:
import time

In [20]:
# batch = next(iter(test_loader))
# data = unpack_data_biked(batch)
# batch[0]

In [21]:
def train(model, dataloader, optimizer, epochs, beta, log_path, compute_elbo = compute_elbo, 
          l1=False, log_interval=1, ccoh_freq = None, is_mmd = False, custom=True):
    
    compute_elbo = compute_elbo_mmd if is_mmd else compute_elbo
    
    model.train()
    
    writer = SummaryWriter(log_path)
    # Lists to store the mean values of metrics per epoch
    mean_losses = []
    mean_log_px_zs = []
    mean_divs = []
    # beta_=beta
    is_beta_cyclical = True if beta ==None else False
    

    
    epochs_time = []
    for epoch in range(epochs):
        start_time = time.time()  # Record start time
        # Lists to store metrics per batch in the current epoch
        batch_losses = []
        batch_log_px_zs = []
        batch_divs = []

        for dataT  in iter(dataloader):
            data = [d.to(device) for d in dataT[0]]
            # data = unpack_data_biked(dataT)
            model(data)
            if l1%2:
                l1=True
            else:
                l1=False
            # mu, L_sigma = model(*data)
            if not is_mmd:
                elbo, log_px_z, div = compute_elbo(model, data, beta,l1=l1)
            else:
                elbo, log_px_z, div = compute_elbo(model, data, beta,l1=l1, custom= custom)
            loss = -elbo
            # print('inside train function loss shape', loss.shape)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Store the metrics for the current batch
            batch_losses.append(loss.item())
            # print(loss.item())
            batch_log_px_zs.append(log_px_z.item())
            batch_divs.append(div.item())
        
        end_time = time.time()  # Record end time
        elapsed_time = end_time - start_time  # Compute elapsed time
        # print(f"Time taken: {elapsed_time:.4f} seconds")
        epochs_time.append(elapsed_time)
        
        # Compute the mean values for the current epoch and store them
        mean_losses.append(sum(batch_losses) / len(batch_losses))
        mean_log_px_zs.append(sum(batch_log_px_zs) / len(batch_log_px_zs))
        mean_divs.append(sum(batch_divs) / len(batch_divs))
        
        
  
        # Log metrics to TensorBoard
        if epoch % log_interval == 0:  # Log every 'log_interval' epochs
            writer.add_scalar('Loss/train', mean_losses[-1], epoch)
            writer.add_scalar('log p(x|z)/train', mean_log_px_zs[-1], epoch)
            writer.add_scalar('DIV/train', mean_divs[-1], epoch)
        if not epoch %10 or epoch==epochs-1:#(epoch!=0 and not epoch %10) or epoch==epochs-1:
            recons_mat = model.self_and_cross_modal_generation([d[:1].to(device) for d in data])
            save_cross_generations(recons_mat, os.path.join(log_path, f'cross_gen_epoch_{epoch}'))
            generation = model.generate_for_calculating_unconditional_coherence(20)
            save_unconditional_generations(generation, os.path.join(log_path, f'uncond_gen_{epoch}'))
            
        if not epoch %50 or epoch==epochs-1:
            model_save_path = os.path.join(log_path, f'model_state_dict_{epoch}.pt')
            torch.save(model.state_dict(), model_save_path)
        # if not epoch %100 or epoch==epochs-1:
        #     fid_path=log_path+'/fid/'
        #     datadir = '/NAS/USERS/e020199/BOMRGD/phd_fouad/mrf_mvae_project/data/PolyMNIST'
        #     mean_fid_randm, mean_fid_condgen, mean_fid_randm_superpos, mean_fid_condgen_superpos = calculate_fid_routine(model, test_loader, params, datadir, fid_path, num_fid_samples= 1000, device='cuda')
        
        # Optional: print the mean metrics for the current epoch
        # print(f"Epoch {epoch+1}/{epochs}, Loss: {mean_losses[-1]:.4f}, log p(x|z): {mean_log_px_zs[-1]:.4f}, DIV: {mean_divs[-1]:.4f} ")
   
    mean_epoch_time = sum(epochs_time)/len(epochs_time)
    print('mean_epoch_time', mean_epoch_time)
    writer.add_scalar('mean_epoch_time', mean_epoch_time)
    writer.close()
    
    return mean_epoch_time, mean_losses, mean_log_px_zs, mean_divs


In [22]:
# batch = next(iter(test_loader))
# i = np.random.randint(batch[0][0].shape[0]-10)
# data = [b[i:10+i].to(device) for b in batch[0]]

In [23]:
# model(data)

In [24]:
# generation = model.generate_for_calculating_unconditional_coherence(10)
# save_unconditional_generations(model.recons, log_path+'data')

## Initialize and train model

In [25]:
# lt= 16
# mu_dim = lt*5
# L_dim = mu_dim*(mu_dim+1)//2
# total_param = mu_dim + L_dim
# total_param

In [26]:
num_workers = 8
hidden_dim=128
n_layers=2
latent_dim=4
diagonal_transf = 'softplus'#'exp'#relu
reduced = False
centered = False


In [27]:
class Params():
    def __init__(self,latent_dim=latent_dim, diagonal_transf =diagonal_transf):
        self.latent_dim = latent_dim
        self.diagonal_transf = diagonal_transf
        self.tmpdir = '../data'
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.device = device
        self.reduced_diag = reduced
        self.centered = centered
        self.inception_module_path = '/NAS/USERS/e020199/BOMRGD/phd_fouad/mrf_mvae_project/data/pt_inception-2015-12-05-6726825d.pth'
        # self.n_layers_vae = n_layers_vae

In [28]:
# # Params SOTA
# my_lt = 16
# lt = 32
# n_modalities = 5
# w_param = z_param = 2*lt
# n_params = n_modalities*(w_param+z_param)
# print('params SOTA ', n_params)

# n = n_modalities*my_lt
# my_n_params = n*(n+1)//2+n
# print('my params ', my_n_params)

#### Train no loop

In [29]:
model = Biked_GMMVAE(Params()).to(device)
# initialize_weights(gmrf_mvae, method='xavier')
# load_path = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/saved_gmrf_models/best_so_far'
# load_path = '../runs/gmrf/resnet/ep_150_beta_0.0001_lt_4_n_2_hd_128_tf_softplus_20250203_140516/model_state_dict_99.pt'
# model.load_state_dict(torch.load(load_path),strict=False)
# model.to(device)

iside latest model
off_diag_scale  0.1


In [30]:
print('cov params ' ,count_parameters(model.off_diag_cov))
for vae in model.modality_vaes:
    print('vae params ' ,count_parameters(vae))
    print('enc params ' ,count_parameters(vae.enc))
    print('dec params ' ,count_parameters(vae.dec))
    break

cov params  39840
vae params  1565647
enc params  840300
dec params  725347


In [31]:
count_parameters(model)

7868305

In [32]:
if False:
    diag_p_data = gmrf_mvae.diag_p.detach().cpu().numpy()
    off_diag_p_data = gmrf_mvae.off_diag_p.detach().cpu().numpy()

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns of subplots

    # First subplot for diag_p
    axs[0].hist(np.exp(diag_p_data), bins=30)
    axs[0].set_title(r'Exp Diag $L_p$')

    # Second subplot for off_diag_p
    axs[1].hist(off_diag_p_data, bins=30)
    axs[1].set_title(r'Off-diag $L_p$')

    plt.tight_layout()  # Adjusts subplot params so that subplots are nicely fit in the figure area
    plt.show()


### Train loop

In [ ]:
lr = 0.0005
batch_size = 256
ccoh_freq = None
# epochs_list = [250, 250, 200, 100]
# betas = [0.0025, 0.001, 0.0005, 0.0001]
# epochs_list = [200]
epochs = 100
betas = [0.0001, 0.0001]
is_mmd = False
custom = True

l1s=[False]
diagonal_transf = 'softplus'
cum_epochs = 0
num_workers = 4

for beta in betas:
    for l1 in l1s:
        params = Params(latent_dim, diagonal_transf)
        model = Biked_GMMVAE(params).to(device)
        train_loader, test_loader = model.getDataLoaders(batch_size = batch_size, num_workers = num_workers, pin_memory = False)
        # initialize_weights(model, method='xavier')
        # load_path = '../runs/gmrf/resnet/best_ep_100_beta_0.0001_lt_4_n_2_hd_128_tf_softplus_20250202_201526/model_state_dict_99.pt'
        # model.load_state_dict(torch.load(load_path),strict=False)   

        optimizer = torch.optim.Adam(model.parameters(), lr = lr)

        log_path = create_log_dir(cum_epochs+epochs, beta, latent_dim, n_layers, hidden_dim, diagonal_transf, l1=l1,
                                 is_mmd=is_mmd, custom=custom)
        mean_epoch_time, mean_losses, mean_log_px_zs, mean_kls = train(model, train_loader, optimizer, epochs=epochs, beta = beta, log_path=log_path, \
                                                      l1=l1, ccoh_freq=ccoh_freq, is_mmd=is_mmd, custom=custom)
        
        loss_save_path = os.path.join(log_path, 'losses')
        
        # fid_path=log_path+'/fid/'
        # datadir = '/NAS/USERS/e020199/BOMRGD/phd_fouad/mrf_mvae_project/data/PolyMNIST'
        # mean_fid_randm, mean_fid_condgen, mean_fid_randm_superpos, mean_fid_condgen_superpos = calculate_fid_routine(gmrf_mvae, test_loader, params, datadir, fid_path, num_fid_samples= 1000, device='cuda')
        # plot_training_curves(mean_losses, mean_log_px_zs, mean_kls, loss_save_path)            

iside latest model
off_diag_scale  0.1


Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init(

Cross-generation grid saved to ..//runs/gmrf/resnet/ep_100_beta_0.0001_lt_4_n_2_hd_128_tf_softplus_20250209_221945/cross_gen_epoch_0
Unconditional generation grid (with superposition) saved to ..//runs/gmrf/resnet/ep_100_beta_0.0001_lt_4_n_2_hd_128_tf_softplus_20250209_221945/uncond_gen_0


Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init() first) or bad device
Error in magma_getdevice_arch: MAGMA not initialized (call magma_init(

In [ ]:
print(error)

In [ ]:
ccoh_freq

In [ ]:
# print(gmrf_mvae.state_dict().keys())

In [ ]:
L.min(), L.max(), L.shape

In [ ]:
# L = gmrf_mvae.get_cholesky().detach().cpu().numpy()
# mu = gmrf_mvae.mu_p
print(L), print(mu)

In [ ]:
if False:
    mean_losses, mean_log_px_zs, mean_kls = train(gmrf_mvae, train_loader, optimizer, epochs, beta = beta, \
                                                  compute_elbo = compute_elbo, l1=False)

In [ ]:
plot_training_curves(mean_losses, mean_log_px_zs, mean_kls)

In [ ]:
# torch.save(gmrf_mvae.state_dict(),'/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/saved_gmrf_models/ep_{}_beta_{}_lt{}_n{}_hd{}_tf_{}_leaky_lr_001'.format(epochs,beta,latent_dim,n_layers,hidden_dim,diagonal_transf))

In [ ]:
# load_path = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/saved_gmrf_models/ep_50_beta_0.001_lt16_n2_hd128_tf_exp_leaky_l2'
# load_path = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/runs/ep_1_beta_0.0025_lt16_n2_hd128_tf_exp_20240321_172209/model_state_dict.pt'
# gmrf_mvae.load_state_dict(torch.load(load_path),strict=False)

In [ ]:
generation = model.generate(20)

In [ ]:
len(generation), generation[0].shape

In [ ]:
data,_ = next(iter(train_loader))
data = [d.to(device) for d in data]
model(data)

In [ ]:
latent_dims = [latent_dim for _ in range(5)]

In [ ]:
latent_dims

In [ ]:
model.qz_x.scale_tril.shape

In [ ]:
qz_x_np = model.qz_x.scale_tril.detach().cpu().numpy()
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
cmap = None  # Placeholder for the first mappable object

for i, q in enumerate(qz_x_np[:5]):
    ax = axes[i]
    # Extract the i-th image from the j-th modality
    mappable = ax.imshow(abs(q))
    if i == 0:
        cmap = mappable  # Save the first mappable object for colorbar
    ax.axis('off')  # Turn off axis
    ax.set_title('$|L_{\Sigma}|$'+str(i+1))

plt.tight_layout()
plt.colorbar(cmap, ax=axes.ravel().tolist())  # Use the first mappable for colorbar
fig.suptitle('Absolute value of $\sqrt{\Sigma}$ for each modality')
plt.show()


In [ ]:
L = model.get_cholesky().detach().cpu().numpy()
plot_covariance_matrix(L, latent_dims, cmap='viridis', grid_color='white', grid_linewidth=2, title = 'L matrix')

In [ ]:
off_diag_l=remove_diagonal_blocks(L,  latent_dims)
plot_covariance_matrix(off_diag_l, latent_dims, cmap='viridis', grid_color='white', grid_linewidth=2, title = 'Off-diag L matrix')

In [ ]:
# Compute number of unnecessary parameters in L_p
epsilon = 0.009
nL = abs(L)/abs(L).max()
print(np.round(nL,3))

oL = np.tril(nL,-1)
mu_size = L.shape[0]
n_lower_params = (mu_size*(mu_size-1))//2 # nuber of lower off-diagonal elements
(n_lower_params - (oL>epsilon).sum()+mu_size)/n_lower_params # Number of parameters < epsilon in absolute value in  the lower matrix

In [ ]:
Sigma = np.matmul(L,L.T)
plot_covariance_matrix(Sigma, latent_dims, off_diag = False, cmap='viridis', grid_color='white', grid_linewidth=2, title = r' $\Sigma$')

In [ ]:
off_diag=remove_diagonal_blocks(Sigma,  latent_dims)
plot_covariance_matrix(off_diag, latent_dims, off_diag = True, cmap='viridis', grid_color='white', grid_linewidth=2, title = r'off diagonal $\Sigma$')

In [ ]:
# scale = .25
# laplace_sample = torch.distributions.laplace.Laplace(0, scale).sample((10000,)).numpy()
# plt.hist(laplace_sample, bins=60)
# plt.title(r'$\mathcal{Laplace}(0,.7)$')
# plt.show()

In [ ]:
# eps = 0.1
# normal_centered = torch.randn(10000,1).numpy()
# plt.hist(normal_centered, bins=60)
# plt.title(r'$\mathcal{N}(0,I)$')
# plt.show()
# normal_centered.shape

In [ ]:
# # Create a boolean mask for the off-diagonal elements
# mask = ~np.eye(L.shape[0], dtype=bool)

# # Use the mask to select the off-diagonal elements
# L_off_diagonal_elements = L[mask]

# eps = 0.1
# flat_off_diag = np.reshape(L_off_diagonal_elements, -1)
# pos_idx = np.where(abs(flat_off_diag)>eps)[0]
# pos_off_diag = flat_off_diag[pos_idx]
# plt.hist(pos_off_diag, bins=60)
# plt.title(r'$L_p$  off- diag values distribution > {}'.format(eps))
# plt.show()

In [ ]:
# eps = 0
# flat_off_diag = np.reshape(off_diag_l, -1)
# pos_idx = np.where(abs(flat_off_diag)>eps)[0]
# pos_off_diag = flat_off_diag[pos_idx]
# plt.hist(pos_off_diag, bins=60)
# plt.title(r'$L_p$ blocks off- diag values distribution > {}'.format(eps))
# plt.show()

In [ ]:
# eps = 0.1
# diag_l =np.diagonal(L)
# plt.hist(diag_l, bins=60)
# plt.title(r'$L_p$ Diag values distribution')
# plt.show()

In [ ]:
# eps = 0
# flat_off_diag = np.reshape(off_diag, -1)
# pos_idx = np.where(abs(flat_off_diag)>eps)[0]
# pos_off_diag = flat_off_diag[pos_idx]
# plt.hist(pos_off_diag, bins=60)
# plt.title(r'$\Sigma_p$ Off-Diag values distribution> {}'.format(eps))
# plt.show()

In [ ]:
# plt.hist(gmrf_mvae.mu_p.detach().cpu().numpy(), bins=60)
# plt.title(r'$\mu_p$ values distribution')
# plt.show()

In [ ]:
# a,_ = next(iter(train_loader))

In [ ]:
# a = [i.numpy() for i in a]

In [ ]:
plot_modalities(generation, n_images=20)

In [ ]:
data, _ = next(iter(train_loader))
plot(data,5)
data = [d[:5].to(device) for d in data]
model(data)

In [ ]:
reconstructions = model.recons
reconstructions = [r.detach().cpu().numpy() for r in reconstructions]
plot(reconstructions, 5)

In [ ]:
n=10
data, _ = next(iter(test_loader))
for j in range(5):
    cond = data[j][:n].to(device) 
    cond_gens = []
    for i in range(5):
        cond_gens.append(model.conditional_generate(cond, i,j , n_sample=1))
    to_plot = [data]+ cond_gens
    plot_cond_modalities(cond, cond_gens, n)

## Classifiers

In [ ]:
print('nb params gmrf mvae ',count_parameters(gmrf_mvae))
for vae in gmrf_mvae.modality_vaes:
    print(vae.modelName ,count_parameters(vae))

In [ ]:
# def load_clf(path):
#     clf = ClfImg()
#     clf.load_state_dict(torch.load(path))
#     return clf
# clf_4 = load_clf('/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/pretrained_img_to_digit_clf_m4')
# clf_3 = load_clf('/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/pretrained_img_to_digit_clf_m3')
# clf_2 = load_clf('/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/pretrained_img_to_digit_clf_m2')
# clf_1 = load_clf('/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/pretrained_img_to_digit_clf_m1')
# clf_0 = load_clf('/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/pretrained_img_to_digit_clf_m0')

In [ ]:
# clfs = [clf_0,clf_1,clf_2,clf_3,clf_4]

In [ ]:
# generations = generate(gmrf_mvae,5)
# generations = [g.detach().cpu() for g in generations]

### Conditional generation

In [ ]:
# n=30
# data, _ = next(iter(test_loader))
# for j in range(5):
#     cond = data[j][:n].to(device) 
#     cond_gens = []
#     for i in range(5):
#         cond_gens.append(gmrf_mvae.conditional_generate(cond, i,j , n_sample=1)) # (cond, idx_i, idx_cond, n_sample=1)
#     plot_cond_modalities(cond, cond_gens, n)

In [ ]:
# test_data,conditional_labels = next(iter(test_loader))
# conditional_labels

accuracies_list = [t. item() for t in accuracies if not np.isnan(t)]
round(np.mean(accuracies_list),2)

In [ ]:
# output_dir = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/cond_generations/'

In [ ]:
np.mean(ssims)

In [ ]:
# get_ccoh(gmrf_mvae, test_loader, clfs, n_iter = 1)

In [ ]:
accuracies = []
ssims = []

for test_data,conditional_labels in tqdm(iter(test_loader)):
    for i in range(5):
        for j in range(5):
            label_classif = clfs[i](test_data[j]).argmax(1)
            cond_gen_ij = model.conditional_generate(test_data[j].to(device), i,j , n_sample=1)
            cond_gen_ij = cond_gen_ij.cpu()
            
            ssims.append(ssim(test_data[j].numpy(),cond_gen_ij.detach().cpu().numpy() , channel_axis =1, data_range = 255.))
            
            generated_classif= clfs[i](cond_gen_ij).argmax(1)
            # First, find where the classifier correctly classifies the test data
            correct_classif = label_classif == conditional_labels

            # Then, filter out the correct classifications from the generated classifications
            correct_generated_classif = generated_classif[correct_classif]

            # Finally, filter out the labels for the correctly classified instances
            correct_conditional_labels = conditional_labels[correct_classif]

            # Now, calculate the accuracy: the ratio of correct generations among these instances
            accuracy = (correct_generated_classif == correct_conditional_labels).float().mean()
            accuracies.append(accuracy)
            print(accuracy.item())
np.mean(accuracies)       

In [ ]:
accuracies = []
ssims = []

for test_data,conditional_labels in tqdm(iter(test_loader)):
    for i in range(5):
        for j in range(5):
            label_classif = clfs[i](test_data[j]).argmax(1)
            cond_gen_ij = gmrf_mvae.conditional_generate(test_data[j].to(device), i,j , n_sample=1)
            cond_gen_ij = cond_gen_ij.cpu()
            
            ssims.append(ssim(test_data[j].numpy(),cond_gen_ij.detach().cpu().numpy() , channel_axis =1, data_range = 255.))
            
            generated_classif= clfs[i](cond_gen_ij).argmax(1)
            # First, find where the classifier correctly classifies the test data
            correct_classif = label_classif == conditional_labels

            # Then, filter out the correct classifications from the generated classifications
            correct_generated_classif = generated_classif[correct_classif]

            # Finally, filter out the labels for the correctly classified instances
            correct_conditional_labels = conditional_labels[correct_classif]

            # Now, calculate the accuracy: the ratio of correct generations among these instances
            accuracy = (correct_generated_classif == correct_conditional_labels).float().mean()
            accuracies.append(accuracy)
            print(accuracy)
np.mean(accuracies)       

In [ ]:
mean_accuracy = 0
for t in accuracies:
    if t ==nan:
        print(t.item())

In [ ]:
# for i, clf in enumerate(clfs):
#     print(clf(test_data[i]).argmax(1)-t)
    

In [ ]:
def compute_cross_coherence():
    # The classifiers might not always be right so we won't count when they're wrong
    for data, target in tqdm(iter(test_loader)):
        

In [ ]:
def reconstruct_and_cross_reconstruct_forw(model, data):
        qu_xs, uss = [], []
        # initialise cross-modal matrix
        px_us = [[None for _ in range(len(model.modality_vae))] for _ in range(len(model.modality_vae))]
        # pw = self.pz(torch.zeros(1, self.params.latent_dim_w), torch.ones(1, self.params.latent_dim_w))
        for m, vae in enumerate(model.modality_vae):
            qu_x, px_u, us = vae(data[m], K=1)
            qu_xs.append(qu_x)
            uss.append(us)
            px_us[m][m] = px_u  # fill-in diagonal
        for e, us in enumerate(uss):
            latents_w, latents_z = torch.split(us, [self.params.latent_dim_w, self.params.latent_dim_z], dim=-1)
            pu = self.pu(*self.pu_params)
            latents_u_to_split = pu.rsample(torch.Size([us.size()[0], us.size()[1]])).squeeze(2)
            latents_w_new, _ = torch.split(latents_u_to_split, [self.params.latent_dim_w, self.params.latent_dim_z], dim=-1)
            us = torch.cat((latents_w_new, latents_z), dim=-1)
            for d, vae in enumerate(self.vaes):
                if e != d:  # fill-in off-diagonal
                    px_us[e][d] = vae.px_u(*vae.dec(us))
        return qu_xs, px_us, uss

In [ ]:
def cross_coherence():
    "Calculates cross-modal coherence"
    model.eval()
    corrs = [[0 for idx, modal in enumerate(model.modality_vaes)] for idx, modal in enumerate(model.modality_vaes)]
    total = 0
    with torch.no_grad():
        for i, dataT in enumerate(test_loader):
            data, targets = unpack_data_polymnist(dataT, device)  # needs to be sent to device
            total += targets.size(0)
            _, px_us, _ = model.reconstruct_and_cross_reconstruct_forw(data)
            for idx_srt, srt_mod in enumerate(model.vaes):
                for idx_trg, trg_mod in enumerate(model.vaes):
                    clfs_results = torch.argmax(clfs[idx_trg](px_us[idx_srt][idx_trg].mean.squeeze(0)), dim=-1)
                    corrs[idx_srt][idx_trg] += (clfs_results == targets).sum().item()
        for idx_trgt, vae in enumerate(model.vaes):
            for idx_strt, _ in enumerate(model.vaes):
                corrs[idx_strt][idx_trgt] = corrs[idx_strt][idx_trgt] / total
        means_target = [0 for idx, modal in enumerate(model.vaes)]
        for idx_target, _ in enumerate(model.vaes):
            means_target[idx_target] = mean(
                [corrs[idx_start][idx_target] for idx_start, _ in enumerate(model.vaes) if idx_start != idx_target])
    return corrs, means_target, mean(means_target)

### Unconditional generation

In [ ]:
def unconditional_coherence(n):
    generations = generate(gmrf_mvae,n)
    generations = [g.detach().cpu() for g in generations]
    classifications = []
    for i, clf in enumerate(clfs):
        classifications.append(clf(generations[i]).argmax(1).numpy())
    # Stack the arrays vertically to create a 2D array
    stacked_arrays = np.vstack(classifications)

    # Check how many columns have all the same values
    num_same_values = np.all(stacked_arrays == stacked_arrays[0], axis=0).sum()

    # Calculate the percentage of columns with the same values
    percentage_same_values = num_same_values / stacked_arrays.shape[1]
    print(percentage_same_values)
    return percentage_same_values

In [ ]:
def unconditional_coherence_relaxed(n):
    generations = generate(gmrf_mvae,n)
    generations = [g.detach().cpu() for g in generations]
    classifications = []
    for i, clf in enumerate(clfs):
        classifications.append(clf(generations[i]).argmax(1).numpy())
    # Stack the arrays vertically to create a 2D array
    stacked_arrays = np.vstack(classifications)

    # Initialize an empty list to store the maximum counts
    max_counts = []

    # Iterate over each index/column
    for col in range(stacked_arrays.shape[1]):
        unique_values, counts = np.unique(stacked_arrays[:, col], return_counts=True)
        max_count = np.max(counts)
        max_counts.append(max_count)

    # Plot a histogram of the maximum counts
    # plt.hist(max_counts, bins=np.arange(1, np.max(max_counts) + 2), align='left', rwidth=0.8)
    # plt.xlabel('Maximum Count of Same Values')
    # plt.ylabel('Frequency')
    # plt.title('Histogram of Maximum Counts of Same classification for Each Modality')
    # plt.show()
    return max_counts

In [ ]:
l = []
for _ in range(100):
    l.append(unconditional_coherence(1000))
np.mean(l)

In [ ]:
plt.hist(l, color = 'green')
plt.title('Unconditional coherence 100 runs 1k generaions')
plt.show()

In [ ]:
counts = []
for _ in tqdm(range(100)):
    counts+= unconditional_coherence_relaxed(1000)
plt.hist(counts, bins=np.arange(1, np.max(counts) + 2), align='left', rwidth=0.8)
plt.xlabel('Maximum Count of Same Values')
plt.ylabel('Frequency')
plt.title('Histogram of Maximum Counts of Same classification for Each Modality')
plt.show()

In [ ]:
(len(test_loader)-1)*128+16

### Compute FID scores

In [ ]:
params = Params()
params.inception_module_path

In [ ]:
fid_path = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/fid'
inception_module_path = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/pt_inception-2015-12-05-6726825d.pth'
datadir = '/NAS/PROJECTS/BOMRGD/phd_fouad/mmvaeplus/data/PolyMNIST'

In [ ]:
mean_fid_randm, mean_fid_condgen

In [ ]:
mean_fid_randm, mean_fid_condgen = calculate_fid_routine(gmrf_mvae, test_loader, params, datadir, fid_path, num_fid_samples= 10000)

In [ ]:
# ##create folders to save the outputs
# for j in [0, 1, 2, 3, 4]:
#         if os.path.exists(os.path.join(fid_path, 'random', 'm{}'.format(j))):
#             shutil.rmtree(os.path.join(fid_path, 'random', 'm{}'.format(j)))
#             os.makedirs(os.path.join(fid_path, 'random', 'm{}'.format(j)))
#         else:
#             os.makedirs(os.path.join(fid_path, 'random', 'm{}'.format(j)))
#         for i in [0, 1, 2, 3, 4]:
#             if os.path.exists(os.path.join(fid_path, 'm{}'.format(j), 'm{}'.format(i))):
#                 shutil.rmtree(os.path.join(fid_path, 'm{}'.format(j), 'm{}'.format(i)))
#                 os.makedirs(os.path.join(fid_path, 'm{}'.format(j), 'm{}'.format(i)))
#             else:
#                 os.makedirs(os.path.join(fid_path, 'm{}'.format(j), 'm{}'.format(i)))

In [ ]:
# for i, data in enumerate(test_loader):
#     data, _ = data
#     data = [d.to(device) for d in data]
#     print(len(data), data[0].shape)
#     gmrf_mvae.reconstruct_for_fid(data, save_dir, i)
#     break